# Riddler Classic Feb 17th 2023 ♦️♠️♣️♥️

[Riddler puzzle page](https://fivethirtyeight.com/features/how-many-rectangles-can-you-make/)

There’s a version of solitaire played in southern Italy with a deck of 40 Neapolitan cards, with `four suits numbered from 1 to 10`. The deck is shuffled and then cards are turned over one at a time. Flipping over the first card you say “one,” the second card “two” and the third card “three.” You repeat this, saying “one” for the fourth card, “two” for the fifth card and “three” for the sixth card. You continue your way through the deck, until you at last say “one” for the 40th card.

If at any point the number you say matches the value of the card you flip over, you lose.

*What is your probability of winning the game?*

## Calcs with pen and paper

In [89]:
from functools import cache
import numpy as np

Let us compute the number of "favorable" cases over the total number of possibilities

In [10]:
nb_full_seq, remainder = divmod(40, 3)
f'View the sequence we say as {"123"*nb_full_seq + "12"[:remainder]} (if we win; else it is interrupted before reaching the 40th card)'

'View the sequence we say as 1231231231231231231231231231231231231231 (if we win; else it is interrupted before reaching the 40th card)'

### Number of total possible cases

The total number of ways to populate the length-40 list is: $\frac{40!}{4!^{10}}$. This is because there are in total factorial(40) ways, to which we then need to discard the suits as all that matters is the number on the card. And so, ten unique numbers repeated four times gets you the denominator, factorial(4) to the power of 10

In [91]:
@cache
def factorial(n: int) -> int:
    """Returns the factorial computed recursively"""
    assert n >= 0
    return 1 if n in (0, 1) else factorial(n-1) * n

nb_total = factorial(40) // (factorial(4) ** 10)
f'Total nb of possibilities: {nb_total}'

'Total nb of possibilities: 12868639981414579848070084500000000'

### Number of favorable cases

A winning sequence is a sequence in which:
- the four 1s were placed on a spot other than 1 in the sequence above ("one, two, three, one, two...")
- the four 2s were placed on a spot other than 2 in the sequence above ("one, two, three, one, two...")
- the four 3s were placed on a spot other than 3 in the sequence above ("one, two, three, one, two...")

(little gotcha: there are 14 '1' spots but 13 '2' and 13 '3' spots)

Let us use binomial coefficients to compute the number of valid such cases:

$\binom{26}{4} * \binom{27}{4} * \binom{27}{4}$

In [26]:
@cache
def binom_coeff(k:int, n:int) -> int:
    'Computes and returns the binomial coefficient "k among n" as the French say'
    assert 0 <= k <= n
    return factorial(n) // (factorial(k) * factorial(n-k))

In [94]:
nb_favorable = binom_coeff(4,26) * binom_coeff(4,27)**2 * factorial(40 - 3 * 4) // (factorial(4)**(10 - 3)) 
f'{nb_favorable=}'

'nb_favorable=306095936727017424205968750000000'

### Getting the answer

In [95]:
answer = nb_favorable / nb_total
answer

0.023786191638673067

## Simulation with Python

In [98]:
SAID = np.tile(np.arange(1,4), 14)[:-2] # the array representing "one, two, three, one, two three"
DECK = np.tile(np.arange(1,11), 4) # an ordered deck (1 to 40 included)

def play() -> bool:
    """Simulates one game and returns whether it is won"""
    np.random.shuffle(DECK)
    return DECK[DECK == SAID].shape[0] == 0

def montecarlo(n_sim: int = 10_000_000) -> float:
    """Runs `n_sim` simulations of the abovecoded game and returns the % of won games"""
    return sum(play() for _ in range(n_sim)) / n_sim

In [ ]:
%%time
montecarlo()

Simulations show a winning rate of `around 0.8 %`

## ChatGPT output, from the exact prompt provided (it's not ready to replace humans yet!)

### First ChatGPT try:

To win the game, you must go through the entire deck without any of the cards you flip over matching the number that you say. The probability of this happening can be calculated using the product rule of probability, which states that the probability of two independent events occurring together is the product of their individual probabilities.

The probability of the first card not matching is 39/40, since there are 39 cards that don't match out of a total of 40 cards. Similarly, the probability of the second card not matching is also 39/40, since one card has already been removed from the deck. The probability of the third card not matching is 38/40, since there are now only 38 cards remaining that don't match. Continuing in this way, the probability of not losing on the 40th card is 1/4, since there are four cards remaining in the deck that would cause you to lose.

So, the probability of winning the game can be calculated as follows:

(39/40) x (39/40) x (38/40) x ... x (2/4) x (1/4) ≈ 0.00039

Therefore, the probability of winning the game is approximately 0.00039, or `0.039%`. In other words, you have a very low chance of winning this version of solitaire.

### Second chatGPT try:

The probability of winning this game can be found by calculating the probability of not losing on each turn, and then multiplying these probabilities together for all 40 turns.

For the first card, there are 4 cards (out of 40) that would cause you to lose, so the probability of not losing on the first turn is 36/40.

For the second card, there are again 4 cards (out of 39) that would cause you to lose, so the probability of not losing on the second turn is 35/39.

Similarly, for the third card, the probability of not losing is 34/38, and so on.

Multiplying all of these probabilities together gives:

(36/40) * (35/39) * (34/38) * ... * (4/8) * (3/7) * (2/6) * (1/5)

This simplifies to:

(3/4) * (7/9) * (17/19) * ... * (1/2) * (3/7) * (1/3) * (1/5)

Using a calculator, this product is approximately 0.128, or about 12.8%.

Therefore, the probability of winning the game is approximately `87.2%`